## Topic Modelling Map Creation
Create maps based on Topics Modelling results for dashboard. Thirty-six maps can be created with this but must be done manually since there's a lot of variation (some location, cuisine, price range combinations have 5 topics instead of 6).

In [1]:
import folium
import re
import fileinput
import pandas as pd
from folium import IFrame
from folium import plugins
import branca.colormap as cm

In [2]:
VEGAS_COORDINATES = (36.1699, -115.1398)
#TORONTO_COORDINATES = (43.6532, -79.3832)
#CALGARY_COORDINATES = (51.0486, -114.0708)
map = folium.Map(location=VEGAS_COORDINATES, zoom_start=11)

In [12]:
pos_df = pd.read_csv('datasets/topic_modelling/pos_bsns_lasvegas_chinese.csv')
neg_df = pd.read_csv('datasets/topic_modelling/neg_bsns_lasvegas_chinese.csv')
pos_df = pos_df[pos_df['RestaurantsPriceRange2'] == 4.0]
neg_df = neg_df[neg_df['RestaurantsPriceRange2'] == 4.0]

In [13]:
pos_topics = list(pos_df)[9:15]
neg_topics = list(neg_df)[9:15]

In [14]:
# read positive topics dataset based on id from negative dataset loop
def get_pos_topics(id):
    positive = pos_df.loc[pos_df['business_id'] == id]
    pos_scores=[0, 0, 0, 0, 0, 0]
    if not positive.empty:
        pos_scores = positive.iloc[0,9:15].tolist()
    return pos_scores

# create html text to format text in map pin popup
def html_text(business_id, stars, neg_scores, name):
    html_topics1 = ['','','','','','']
    html_topics2 = ['','','','','','']
    
    pos_scores = get_pos_topics(business_id)
    for m in range(0, len(pos_topics)):
        html_topics1[m] = '{:18}: {:.2f}%'.format(pos_topics[m], pos_scores[m]*100)
    for n in range(0, len(neg_topics)):
        html_topics2[n] = '{:18} {:.2f}%'.format(neg_topics[n], neg_scores[n]*100)
        
    pos_topics_text = 'Positive Topics<br>' + '<br>'.join(html_topics1)
    neg_topics_text = 'Negative Topics<br>' + '<br>'.join(html_topics2)
    stars_text = '<h3> Stars: ' + '{:.2f}'.format(stars) + '</h3>'
    name_text = '<h3> Name: ' + name + '</h3>'
    return name_text + stars_text + '<p>' + pos_topics_text + '<br><br>' + neg_topics_text + '</p>'

# create folium Iframe & Popup by creating html
def get_popup(business_id, stars, neg_scores, name):
    html = "" + html_text(business_id, stars, neg_scores, name) + ""
    iframe = folium.IFrame(html=html, width=300, height=200)
    return folium.Popup(iframe, max_width=1000)

In [6]:
# set map pin color based on star rating
def star_color(stars):
    if stars <= 1.5:
        return '#FF0000'
    elif stars <= 2.5:
        return '#FFAD00'
    elif stars <= 3.5:
        return '#FFFF00'
    elif stars < 4.5:
        return '#A1FF00'
    else:
        return '#007200'
    
def get_icon(stars):
    return plugins.BeautifyIcon(border_color='transparent', background_color=star_color(stars), 
                                icon='circle', icon_shape='marker', text_color='#FFF')
    

In [7]:
# get star rating from positive topic data set
def get_star(id, stars):
    positive = pos_df.loc[pos_df['business_id'] == id]
    pos_star = stars
    if not positive.empty:
        pos_star = positive['stars'].values[0]
    return pos_star

In [23]:
for i, row in enumerate(neg_df.itertuples(), 0):
    print(i)
    probs = neg_df.iloc[i,9:15].tolist()
    pos_stars = get_star(row.business_id, row.stars)
    avg_stars = (row.stars + pos_stars)/2 # get average star rating of negative and positive dataset
    folium.Marker(location=[row.latitude, row.longitude], popup=get_popup(row.business_id, avg_stars, probs, row.name), 
                  icon=get_icon(avg_stars)).add_to(map)
    if row.Index > 600:
        break

0
1
2
3
4
5


In [932]:
colormap = cm.LinearColormap(
    ['red', 'orange', 'yellow', 'green'],
    vmin=0, vmax=5
)
colormap.caption = 'Star Rating'
map.add_child(colormap)

In [933]:
map.save('maps/lasvegas_american_4.html')